In [1]:
import cv2
import numpy as np
import math

# Function to detect hand gestures
def detect_shape(contour):
    perimeter = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)

    if len(approx) == 3:
        return "Triangle"
    elif len(approx) == 4:
        return "Square"
    else:
        return "Circle"

# Function to track finger and determine shape
def track_finger(frame, contour):
    # Calculate centroid of the contour
    M = cv2.moments(contour)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])

    # Calculate distance between centroid and finger tip
    finger_tip_x = contour[0][0][0]
    finger_tip_y = contour[0][0][1]
    distance = math.sqrt((cx - finger_tip_x) ** 2 + (cy - finger_tip_y) ** 2)

    # Determine shape based on distance
    if distance < 50:
        return "Circle"
    elif distance < 100:
        return "Square"
    else:
        return "Triangle"

# Main function
def main():
    # Open the camera
    cap = cv2.VideoCapture(0)

    while True:
        # Read frame from the camera
        ret, frame = cap.read()

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Apply thresholding to create binary image
        _, threshold = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV)

        # Find contours in the binary image
        contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Iterate through each contour
        for contour in contours:
            # Ignore small contours
            if cv2.contourArea(contour) > 1000:
                # Detect shape
                shape = detect_shape(contour)

                # Track finger and determine shape
                tracked_shape = track_finger(frame, contour)

                # Draw contour and shape label on the frame
                cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
                cv2.putText(frame, tracked_shape, (contour[0][0][0], contour[0][0][1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                # Draw rectangle, square, or circle based on shape
                if tracked_shape == "Rectangle":
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                elif tracked_shape == "Square":
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                elif tracked_shape == "Circle":
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    center = (int(x), int(y))
                    radius = int(radius)
                    cv2.circle(frame, center, radius, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow("Hand Gesture Recognition", frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close all windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

KeyboardInterrupt: 